<a href="https://colab.research.google.com/github/Nelkit/36103-AT2-data-analysis-project/blob/main/36103_AT2B_data_analysis_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assessment task 2: Data analysis project

---



---
## 0. Setup Environment

### 0.a Install Mandatory Packages

> Do not modify this code before running it

In [ ]:
# Do not modify this code

import os
import sys
from pathlib import Path

COURSE = "36103"
ASSIGNMENT = "AT2"
DATA = "data"

asgmt_path = f"{COURSE}/assignment/{ASSIGNMENT}"
root_path = "./"

if os.getenv("COLAB_RELEASE_TAG"):

    from google.colab import drive
    from pathlib import Path

    print("\n###### Connect to personal Google Drive ######")
    gdrive_path = "/content/gdrive"
    drive.mount(gdrive_path)
    root_path = f"{gdrive_path}/MyDrive/"

print("\n###### Setting up folders ######")
folder_path = Path(f"{root_path}/{asgmt_path}/") / DATA
folder_path.mkdir(parents=True, exist_ok=True)
print(f"\nYou can now save your data files in: {folder_path}")

if os.getenv("COLAB_RELEASE_TAG"):
    %cd {folder_path}



###### Connect to personal Google Drive ######
Mounted at /content/gdrive

###### Setting up folders ######

You can now save your data files in: /content/gdrive/MyDrive/36103/assignment/AT2/data
/content/gdrive/MyDrive/36103/assignment/AT2/data


### 0.b Disable Warnings Messages

> Do not modify this code before running it

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### 0.c Install Additional Packages

> If you are using additional packages, you need to install them here using the command: `! pip install <package_name>`

In [ ]:
!pip install scipy

### 0.d Import Packages

In [ ]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import matplotlib.gridspec as gridspec

### 0.f Reusable Functions

## 1. Introduction

In [ ]:
# @title 1.1 Project Description
project_description = ""

wgt_project_description = widgets.Textarea(
    value=None,
    placeholder=project_description,
    description='Project Description:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(height="100%", width="auto")
)
wgt_project_description

Textarea(value='', description='Project Description:', layout=Layout(height='100%', width='auto'), placeholder…

In [ ]:
# @title 1.2 Business Objective
business_objective = ""

wgt_business_objective = widgets.Textarea(
    value=None,
    placeholder=business_objective,
    description='Business Objective:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(height="100%", width="auto")
)
wgt_business_objective

Textarea(value='', description='Business Objective:', layout=Layout(height='100%', width='auto'), placeholder=…

## 2. Data Loading and Overview

## 3. Exploratory Data Analysis (EDA)

### 3.1 Explore features

### 3.2 Explore target variable

## 4. Data Preprocessing

## 5. Feature Engineering

## 6. Data Modeling

## 7. Model Evaluation

## 8. Insights and Conclusions